 ## 9.1 ROOM - podstawy

Przyjrzyjmy się `ROOM` - jest to warstwa abstrakcji służąca jako punkt dostępu do do bazy `SQLite`, znacznie upraszcza pracę z wewnętrzną bazą na urzadzeniu. W tej aplikacji zaimplementujemy bazę przechowującą słowa - umożliwimy wyświetlanie wszystkich słów oraz dodanie nowego elementu. Wykorzystamy elementy architektury - `ViewModel`, `LiveData` oraz `ROOM`. Przykłado pochodzi z oficjalnego [codelab](https://developer.android.com/codelabs/android-room-with-a-view-kotlin#7).

<img src="https://media3.giphy.com/media/MMcJ5i46Q0FIBspUEQ/giphy.gif" width="200" />

Rozpocznijmy od dodania odpowiednich zależności

In [ ]:
plugins {
    id 'com.android.application'
    id 'org.jetbrains.kotlin.android'
    id 'kotlin-android'
    id 'kotlin-kapt'
}

// ROOM
implementation("androidx.room:room-ktx:2.4.3")
kapt("androidx.room:room-compiler:2.4.3")

// ViewModel
implementation 'androidx.lifecycle:lifecycle-viewmodel-ktx:2.5.1'
// LiveData
implementation "androidx.lifecycle:lifecycle-livedata-ktx:2.5.1"

// Activity
implementation "androidx.activity:activity-ktx:1.5.1"

Nasza aplikacja składała się tylko z jednej aktywności, na której dodamy `RecyclerView` - tym razem wykorzystamy `ListAdapter` (zamiast zwykłego adaptera) oraz `DiffUtil` do określenia różnic między elementami kolekcji.

### **ROOM**

#### **Entity**

Tworzenie bazy rozpoczniemy od klasy `Word` jest to byt (`Entity`) reprezentujący tabelę w bazie `SQLite`.

In [ ]:
@Entity(tableName = "word_table")
class Word()

`@Entity(tableName = "word_table")` - każda klasa oznaczona adnotacją `@Entity` reprezentuje tabelę `SQLite`. Możemy określić nazwę tabeli jaką chcemy nadać w samej bazie wykorzystując właściwość `tableName` - tylko w przypadku gdy chcemy aby nazwa tabeli była inna niż nazwa klasy.

Nasza klasa będzie posiadała jedną właściwość (pole), będzie to słowo (`word`) typu `String`. Każda właściwość w klasie reprezentuje kolumnę w tabeli - więc w naszym przykładzie zostanie utworzona jedna tabela o nazwie `word_table` posiadająca jedną kolumnę o nazwie `word`.

In [ ]:
@Entity(tableName = "word_table")
class Word(@PrimaryKey @ColumnInfo(name = "word") val word: String)

- `@PrimaryKey` - każda `Entity` wymaga posiadania jednego klucza głównego (unikalnego identyfikatora), w tym przykładzie, dla uproszczenia, samo słowo będzie jednocześnie kluczem głównym.
- `@NonNull` - oznacza że parametr, pole lub wartość zwracana metody nie może mieć wartości `null`
- `@ColumnInfo(name = "word")` - pozwala nadać nazwę kolumnie, jeżeli jest ona inna niż nazwa pola - w tym przykładzie jest ona taka sama, więc tą adnotację możemy usunąć.

#### **DAO**

`DAO` (*Data Access Object*) weryfikuje kod `SQL` podczas kompilacji i powiązuje go z odpowiednią metodą - musi być interfejsem lub klasą abstrakcyjną.

In [ ]:
@Dao
interface WordDao {

Podobnie jak przy `@Entity` posługujemy się adnotacją `@Dao` do oznaczenia interfejsu jako części `ROOM`. Tutaj dodamy kilka metod pozwialających nam dodać element do bazy, usunąć wszystkie elementy oraz wyciągnąć z bazy listę posortowanych elementów. Rozpocznijmy od metody `insert`

In [ ]:
    @Insert(onConflict = OnConflictStrategy.IGNORE)
    suspend fun insert(word: Word)

- `suspend fun insert(word: Word);` deklaruje funkcję zawieszenia wstawiającą jeden element
- `@Insert` - specjalna adnotacja, która nie wymaga podawania wprost kodu `SQL` - istanieją jeszcze `@Delete` oraz `@Update` - mocno ułatwia to wykonywanie podstawowych operacji na bazie
- `onConflictStrategy` - określa zachowanie w przypadku wystąpienia konfliktu podczas próby dodania elementu do bazy (czyli próby dodania elementu który już istnieje) - tutaj będziemy ignorować, czyli próba dodania słowa które już istnieje w bazie zostanie zignorowana

Kolejną będzie metoda usuwająca wszystkie elementy z listy

In [ ]:
    @Query("DELETE FROM word_table")
    suspend fun deleteAll()

Adnotacja `@Delete` służy do usuwania tylko jednego elementu, do usunięcia wszystkich musimy dodać odpowiednie zapytanie.
- `@Query` - wymaga podania zapytania SQL jako parametru typu `String`

Ostatnią metodą będzie `getAlphabetizedWords` zwracająca posortowaną listę wszystkich słów.

In [ ]:
    @Query("SELECT * FROM word_table ORDER BY word ASC")
    fun getAlphabetizedWords(): LiveData<List<Word>>

Metoda zwraca listę wszystkich słów jako `LiveData`

#### **RoomDatabase**

Jedną z ważniejszych cech bazy `ROOM` jest brak zezwolenia na wykonywanie zapytań na wątku głównym (nazywanym również wątkiem ui). Wszystkie zapytania zwracające dane z bazy są wykonywane asynchronicznie. Klasa reprezentująca samą bazę musi być **klasą abstrakcyjną** i musi rozszerzać klasę `RoomDatabase`.

In [ ]:
@Database(entities = [Word::class], version = 1, exportSchema = false)
abstract class WordRoomDatabase : RoomDatabase() {

Ponownie wykorzystujemy adnotację aby poinformować kompilator że ta klasa jest komponentem `ROOM`. Sama adnotacja `@Database` przyjmuje kilka parametrów
- `entities` - określa byty (tabele) należące do bazy
- `version` - określa wersję bazy
- `exportSchema` - migracje baz danych tutaj pominiemy, więc ustawimy na `false`

Baza wystawia `DAO` przez abstrakcyjny getter dla każdego `@Dao`.

In [ ]:
    abstract fun wordDao(): WordDao

Samą bazę zaimplementujemy jako `Singleton` aby zapobiec tworzeniu wielu instancji bazy. Innymi słowy chcemy zapobiec wielokrotnemu otwarciu bazy i niekontrolowanemu wykonywaniu operacji.

W pierwszym kroku dodajmy zmienną `INSTANCE` w której będziemy przechowywać naszą jedyną instancję teggo obiektu

In [ ]:
    companion object {
        @Volatile
        private var INSTANCE: WordRoomDatabase? = null

Potrzebujemy jeszcze metodę `getDatabase`, która zainicjuje nam bazę jeżeli ta jeszcze nie istnieje, lub zwróci nam `INSTANCE` jeżeli została utworzona wcześniej.

In [ ]:
    fun getDatabase(context: Context): WordRoomDatabase {
        return INSTANCE ?: synchronized(this) {
            val instance = Room.databaseBuilder(
                context.applicationContext,
                WordRoomDatabase::class.java,
                "word_database_kotlin"
            ).build().also { INSTANCE = it }
            instance
        }
    }

W tym celu wykorzystujemy `Room.databaseBuilder` który przyjmuje kontekst. Zwróćmy uwagę na zastosowanie kontekstu aplikacji, a nie np. aktywności - kontekst aplikacji jest najszerszym kontekstem i najczęściej to na nim tworzymy bazę i wszystkie powiązane z nią elementy.

### **ViewModel**

Utwórzmy `ViewModel` - tym razem rozszerzymy `AndroidViewModel`, a nie jak poprzednio `ViewModel` - różnica jest w parametrze. `AndroidViewModel` może przekazać parametr - `RoomDatabase` wymaga podania kontekstu (zazwyczaj jest nim `Application`), który przekazujemy jako parametr. Dodajmy też dwa pola reprezentujące bazę danych oraz `LiveData` naszej listy słów.

In [ ]:
class WordViewModel(application: Application) : AndroidViewModel(application) {
    private val db: WordRoomDatabase
    val getAllWords: LiveData<List<Word>>

Dodajmy również blok `init`

In [ ]:
    init {
        db = WordRoomDatabase.getDatabase(application)
        getAllWords = db.wordDao().getAlphabetizedWords()
    }

Ostatnią metodą klasy będzie `insert` umożliwiająca dodanie elementu do bazy - wykorzystamy `Coroutines` z `viewModelScope`

In [ ]:
    fun insert(word: Word) {
        viewModelScope.launch{
            db.wordDao().insert(word)
        }
    }

### **RecyclerView + ListAdapter + DiffUtil**

Przejdźmy do wyświetlenia danych i umożliwienia dodania nowego słowa. Wpierw zdefiniujmy layouty. Layout głównej aktywności zawierający `RecyclerView`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout
    xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".view.MainActivity">

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/recyclerview"
        android:layout_width="0dp"
        android:layout_height="0dp"
        tools:listitem="@layout/recyclerview_item"
        android:padding="12dp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintLeft_toLeftOf="parent"
        app:layout_constraintRight_toRightOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <com.google.android.material.floatingactionbutton.FloatingActionButton
        android:id="@+id/fab"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_margin="16dp"
        android:contentDescription="add word"
        android:src="@drawable/ic_add"/>

</androidx.constraintlayout.widget.ConstraintLayout>

Dodajmy również layout dla pojedynczego elementu listy

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    android:orientation="vertical"
    android:layout_width="match_parent"
    android:layout_height="wrap_content">

    <TextView
        android:id="@+id/textView"
        android:layout_width="match_parent"
        android:layout_height="wrap_content" />
</LinearLayout>

Rozpocznijmy od utworzenia klasy `ViewHolder`

In [ ]:
class WordViewHolder(itemView: View) : RecyclerView.ViewHolder(itemView) {
    private val wordItemView: TextView = itemView.findViewById(R.id.textView)

    fun bind(text: String?) {
        wordItemView.text = text
    }
}

Następnie Dodajmy klasę `WordComparator` rozszerzjącą klasę `DiffUtil.ItemCallback`, służy ona dwóm celom
- indeksowanie list
- rozróżnianie elementów

Mamy do zaimplementowania dwie metody
- `areItemsTheSame` - wskazująca czy dwa elementy są tym samym (np. porównanie referencyjne)
- `areContentsTheSame` - wskazująca czy dwa elementy są takie same (np. porównanie strukturalne)

In [ ]:
class WordsComparator : DiffUtil.ItemCallback<Word>() {
    override fun areItemsTheSame(oldItem: Word, newItem: Word): Boolean {
        return oldItem === newItem
    }

    override fun areContentsTheSame(oldItem: Word, newItem: Word): Boolean {
        return oldItem.word == newItem.word
    }
}

Pozostaje implementacja `WordListAdapter` rozszerzającej `ListAdapter` - jest to klasa dostarczająca taką samą funkcjonalność jak `RecyclerView.Adapter`, dodatkowo umożliwia obliczenie różnic między listami (na wątku roboczym)

In [ ]:
class WordListAdapter(wordsComparator: WordsComparator) : ListAdapter<Word, WordViewHolder>(wordsComparator) {

    override fun onCreateViewHolder(parent: ViewGroup, viewType: Int): WordViewHolder {
        return WordViewHolder(LayoutInflater.from(parent.context).inflate(
            R.layout.recyclerview_item, parent, false
        ))
    }

    override fun onBindViewHolder(holder: WordViewHolder, position: Int) {
        val current = getItem(position)
        holder.bind(current.word)
    }
}

### **Aktywność**

Do głównej aktywności dodajmy dwie zmienne

In [ ]:
    private val wordViewModel: WordViewModel by viewModels()

    private var num = 0 // licznik słów

Dodajmy `Recyclerview`

In [ ]:
val recyclerView = findViewById<RecyclerView>(R.id.recyclerview)
val adapter = WordListAdapter(WordsComparator())
recyclerView.adapter = adapter
recyclerView.layoutManager = LinearLayoutManager(this)

Podłączmy obserwator do listy słów

In [ ]:
wordViewModel.getAllWords.observe(this) { words ->
    words?.let { adapter.submitList(it) }
}

Gdy pojawia się nowa wersja listy wywołujemy metodę `submitList` klasy `ListAdapter` - lista zostaje automatycznie odświeżona przy każdej zmianie.

Na koniec dodajmy możliwość dodania nowego słowa

In [ ]:
val fab = findViewById<FloatingActionButton>(R.id.fab)
fab.setOnClickListener {
    wordViewModel.insert(Word("word $num"))
    num++
}

Możemy przetestować aplikację

<img src="https://media3.giphy.com/media/MMcJ5i46Q0FIBspUEQ/giphy.gif" width="150" />